<a href="https://colab.research.google.com/github/Andres8bit/parallel-computing/blob/main/quad_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-8x9fxpw5
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-8x9fxpw5
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=1030e8d46e59ecb44171202b25a976d826d43c3685fda1ff1e9ab6d6d55ccf57
  Stored in directory: /tmp/pip-ephem-wheel-cache-xkso3kwa/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
Parallel Quadtree Implementation:
    Uses dynamic Parallelism to implement a recursive quadtree kernel.
    A Quadtrees work by splittinga 2D space by subdividing it into 4 parts.
    Each quadrant is a noode of our tree, and contains points. If the number
    of points within a region is above a threshold (2 for this example), the region is further split
    into 4 child nodes/regions. This repeats recursively until no more splits are possible.
    Each region/node is assigned to a thread-block. At depth = 0, only one block is assigned to all points,our root node.
    The root node then lauches 4 children nodes/blocks. Inorder to subdivide each region an calculate the number of nodes
    per region the center of the bounding box that surrounds each region is calculated.
    A scan consisting of 4 elements is done inorder to compute the offsets to the locations where the points are to be stored.
    Points are then reordered so that they are grouped together by region. We 
    use two buffers. The buffers hold all our points. At depth = 0, or at our first step,
    one buffer is empty while the other holds our points. Durring each step we create 4 new thread-blocks.
    Each of which point to portions of our buffer. With each thread-block pointing to a subset of the buffer corresponding to the total number of points contained within each region.
    We then reorder our points and copy them into buffer2, so that they fit within the correct subsection of our buffer.
    This process is repeated  back and forth durring each split.




    Point Class -> defines basic 2D point class    
              methods: Point() -> constructs empty point object
                       Point(float x,float y) -> constructor
                       get_point(int id) -> returns the point corresponding to
                                            id returns point as float2.
                       set_point(int id, const float2 point) -> sets point val
                                                                at id. 
                                                                Returns void.
    
    Bounding_box Class ->

    Quad_tree Class ->

    Parameters Struct ->

In [20]:
%%cu
#include <iostream>

class Point{
   private:
    float *x;
    float *y;
    
    public:
    __host__ __device__ Point(): x(NULL), y(NULL){}
    
    __host__ __device__ Point(float *x_val, float *y_val): x(x_val), y(y_val){}

    __host__ __device__ __forceinline__ float2 get_point(int id){
        return make_float2(x[id],y[id]);
    }

    __host__ __device__ __forceinline__ void set_point(int id,const float2 point)
    {
        x[id] = point.x;
        y[id] = point.y;
    }
};

class Bounding_box{
    private:
     float2 min_point;
     float2 max_point;
    
    public:
     __host__ __device__ Bounding_box(){
         min_point = make_float2(0.0,0.0);
         max_point = make_float2(1.0,1.0);
     }

     __host__ __device__ void compute_center(float2 &center)const{
         center.x = 0.5*(min_point.x + max_point.x);
         center.y = 0.5*(min_point.y + max_point.y);
         
     }

     __host__ __device__ __forceinline__ const float2 get_max()const{
         return max_point;
     }

     __host__ __device__ __forceinline__ const float2 get_min()const{
         return min_point;
     }

     __host__ __device__ bool contains(const float2 &p)const{
         return p.x >= min_point.x && p.x < max_point.x && 
                p.y >= min_point.y && p.y < max_point.y;
     }

     __host__ __device__ void set(float min_x,float min_y,float max_x,float max_y)
     {
         min_point.x = min_x;
         min_point.y = min_y;
         max_point.x = max_x;
         max_point.y = max_y;
     }
};

class Quadtree_node{
    private:
      int my_id;
      int start;
      int end;
      Bounding_box box;

    public:
      __host__ __device__  Quadtree_node() : my_id(0),start(0),end(0){}

      __host__ __device__ int id()const{
          return my_id;
      } 

      __host__ __device__ void set_id(int new_id){
          my_id = new_id;
      }

      __host__ __device__ __forceinline__ const Bounding_box &bounding_box()const{
          return box;
      }

      __host__ __device__ __forceinline__ void set_bounding_box(float min_x,
                                            float min_y,float max_x,float max_y){

        box.set(min_x,min_y,max_x,max_y);                                            
    }

    __host__ __device__ __forceinline__ int num_points()const{
        return end-start;
    }

    __host__ __device__ __forceinline__ int points_begin()const{
        return start;
    }
    
    __host__ __device__ __forceinline__ int points_end()const{
        return end;
    }

    __host__ __device__ __forceinline__ void set_range(int begin,int stop){
      start = begin;
      end = stop;
    }
};


struct Parameters{
    int point_selector;
    int num_nodes_cur_level;
    int depth;
    const int max_depth;
    const int min_points_per_nodes;

    __host__ __device__ Parameters(int max_depth,int min_points_per_nodes):
        point_selector(0),
        num_nodes_cur_level(1),
        max_depth(max_depth),
        min_points_per_nodes(min_points_per_nodes){}

    __host__ __device__ Parameters(const Parameters &params):
        point_selector((paras.point_selector + 1) % 2),
        num_nodes_cur_level(4*parames.num_nodes_cur_level),
        depth(params.depths + 1),
        min_points_per_nodes(params.min_points_per_nodes){}
};

__device__ bool check_num_points_and_depth(Quadtree_node &node, Points *poitns,
                                           int num_points,Parameters params)
{

        if (params.depth > = params.max_depth 
            || num_points <= params.min_points_per_nodes){
            if (params.point_selector == 1){
              int itr = node.points_begin();
              int end = node.points_end();
              for (itr += threadIdx.x; itr < end; it += blockDim.x)
                if (itr < end)
                    points[0].set_point(it,points[1].get_point(itr));
            }
            return true;
          }             
          return false;                                  
}

__device__ void count_points_in_children(const Points &in_points,int* smem,
                                         int range_begin,int range_end, float2 center)
{
    if (threadIdx.x < 4) smem[threadIdx.x] = 0;
    __syncthreads();

    for (int itr = range_begin + threadIdx.x; itr , range_end; itr += BlockDim.x){
        float2 p = int_points.get_point(itr);
        if (p.x < center.x && p.y >= center.y)
            atomicAdd(&smem[0],1);
        if (p.x >= center.x && p.y >= center.y)
            atomicAdd(&smem[1],1);
        if (p.x < center.x && p.y < center.y)
            atomicAdd(&smem[2],1)
        if (p.x >= center.x &7 p.y < center.y)
            atomicAdd(&smem[3],1);
    }    
    __syncthreads();
}

__device__ void scan_for_offsets(int node_points_begin,int* smem){
    int* smem2 = &smem[4];
    if (threadIdx.x == 0){
        for (int i = 0; i < 4; i++)
          smem2[i] = i == 0 ? 0 : smem2[i-1] + smem[i-1];
        for (int i = 0; i < 4; i++)
          smem2[i] += node_points_begin;
    }
    __syncthreads();
}

__device__ void reorder_points(Points &out_points,const Points &in_points,
                        int *smem,int range_begin,int range_end, float2 center)
{
    int* smem2 = &smem[4];
    int dest = 0;
    float2 p = {0,0};
 
    for (int itr = range_begin + threadIdx.x; itr < range_end; itr += blockDim.x){
         dest = 0;
         p = in_points.get_point(itr);
         if (p.x < center.x && p.y >= center.y)
              dest = atomicAdd(&smem2[0],1);
         if (p.x >= center.x && p.y >= center.y)
              dest = atomicAdd(&smem2[1],1);
         if (p.x < center.x && p.y < center)
              dest = atomicAdd(&smem2[2],1);
         if (p.x >= center.x && p.y < center.y)
               dest = atomicAdd(&smem2[3],1);
         
         out_points.set_point(dest,p);
    }
    __syncthreads();
}

__device__ void prepare_children(Quadtree_node* children,Quadtree_node& node,
                                 const Bounding_box& box,int* smem)
{
    int child_offset = 4*node.id();

    children[child_offset + 0].set_id(4 * node.id() + 0);
    children[child_offset + 1].set_id(4 * node.id() + 4);
    children[child_offset + 2].set_id(4 * node.id() + 8);
    children[child_offest + 3].set_id(4 8 node.id() + 12);
    
    const float2 &min_point = box.get_min();
    const float2 &max_point = box.get_max();
 
    children[child_offset + 0].set_bounding_box(min_point.x,center.y,
                                                center.x,max_point.y);
 
    children[chiid_offset + 1].set_bounding_box(center.x,center.y,
                                                max_point.x,max_point.y);
 
    children[child_offset + 2].set_bounding_box(min_point.x,min_point.y,
                                                center.x,center.y);
 
    children[child_offset + 3].set_bounding_box(center.x,min_point.y,
                                                max_point.x,center.y);
 

    children[child_offset + 0].set_range(node.points_begin(), smem[4 + 1]);
    children[child_offset + 1].set_range(smem[4 + 0],smem[4 + 1]);
    children[child_offset + 2].set_range(smem[4 + 1],smem[4 + 2]);
    children[child_offset + 3].set_range(smem[4 + 2],smem[4 + 3]);
 
}

__global__ void build_quadtree_kernel(Quadtree_kernel *nodes, Points* poitns,Parameters params)
{
    __shared__ int smem[8];
    
    Quadtree_node &node = nodes[blockIdx.x];
    node.set_id(node.id() + blockIdx.x);
    int num_points = node.num_points();
    bool exit = check_num_points_and_depth(node,points,num_points,params);
    
    if (exit) return;
    
    const Bounding_box &box = node.bounding_box();
    float2 center;
    box.compute_center(center);
 
    int range_begin = node.points_begin();
    int range_end = node.points_end();
    const Points &in_points = points[params.point_selector];
    Points &out_points = points[(params.point_selector + 1)];
     
    count_points_in_children(int_points,smem,range_begin,range_end,center);
    
    scan_for_offsets(node.points_begin(),smem);
 
    reorder_points(out_points,in_points,smem,range_begin,range_end,center);
 
    if (threadIdx.x == blockDim.x - 1){
        Quadtree_node* children = &nodes[params.num_nodes_at_this_level];

        prepare_children(children,node,box,smem);

        build_quadtree_kernel<<<4, blockDim.x, 8 * sizeof(int)>>>
                              (children,points,Parameters,true);
    }
}
int main(){
    
    return 0;
}